# Installations

In order to access the YOLO model, we need to first install Ultralytics.

In [6]:
%pip install ultralytics
%pip install opencv-python
%pip install sklearn
%pip install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not wr

# Image Registration (optional)

In [7]:
import os
import cv2
import shutil
from pathlib import Path

# Paths
root_dir = Path("/Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/WiSARD_dataset/registered")
dataset_splits = ["train", "val", "test"]

# Image registration function (thermal aligned to RGB)
def register_thermal_to_rgb(rgb_path, thermal_path, output_path):
    rgb_img = cv2.imread(str(rgb_path), cv2.IMREAD_COLOR)
    thermal_img = cv2.imread(str(thermal_path), cv2.IMREAD_GRAYSCALE)

    # Resize thermal to match RGB if needed
    thermal_resized = cv2.resize(thermal_img, (rgb_img.shape[1], rgb_img.shape[0]))

    # Save aligned thermal image
    output_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(output_path), thermal_resized)

# Copy labels
for split in dataset_splits:
    rgb_dir = root_dir / split / "rgb"
    thermal_dir = root_dir / split / "thermal"
    labels_dir = root_dir / split / "labels"

    # Create output labels dir
    labels_dir.mkdir(parents=True, exist_ok=True)

    for rgb_img_path in rgb_dir.glob("*.jpg"):
        base_name = rgb_img_path.stem

        # 1. Register thermal to RGB
        thermal_img_path = thermal_dir / f"{base_name}.jpg"
        if thermal_img_path.exists():
            register_thermal_to_rgb(rgb_img_path, thermal_img_path, thermal_img_path)

        # 2. Copy label file from source labels folder
        source_label = Path(f"data/{split}/labels/{base_name}.txt")  # adjust if your original labels are elsewhere
        dest_label = labels_dir / f"{base_name}.txt"
        if source_label.exists():
            shutil.copy(source_label, dest_label)

print("Thermal registration and label copying complete.")

Thermal registration and label copying complete.


# Import Model

From Ultralytics, now we can import the YOLO model.

In [2]:
from ultralytics import YOLO
import torch

/Users/ikmalbasirun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
print(torch.backends.mps.is_available())  # Should print: True

True


YOLO has different types of model, so we need to specify which version we would like to load. In this case, we will be using a **pre-trained** yolo-version-11.

In [6]:
# Load a model
modelv5_tuning = YOLO("yolov5nu.pt")   # these first three models are for hyperparameter tuning
modelv8_tuning = YOLO("yolov8n.pt")
modelv11_tuning = YOLO("yolo11n.pt")

modelv5 = YOLO("yolov5nu.pt")   # these last 3 are for training the final model, which uses the best hyperparameters
modelv8 = YOLO("yolov8n.pt")
modelv11 = YOLO("yolo11n.pt")
# I've added these last three to ensure that final models dont continue training on one of the hyperparameter tuning models

#using MPS instead of CPU
device = "mps" if torch.backends.mps.is_available() else "cpu"

We're training our model via a YAML file. We would have used a csv file, but this model does not recognise this. YAML files are unfortunately more complicated, so we will discuss this further during the meeting.

# YOLOv5

## Hyperparameter Tuning

### Tuner

In [19]:
# Define search space
search_space = {
    "lr0": (1e-5, 1e-2),
    "momentum": (0.85, 0.98)
}

# Tune hyperparameters on WISARD DATASET
modelv5.tune(
    data="wisard_dataset_rgb.yaml",
    epochs=2,
    iterations=15,
    imgsz=512,
    batch=16,
    optimizer="Adam",
    space=search_space,
    device="mps",
    workers=8
)

Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune23'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/15 with hyperparameters: {'lr0': 0.01, 'momentum': 0.937}


/Users/ikmalbasirun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Transferred 427/427 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.126 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 MPS (Apple M4 Pro)
engine/trainer: task=detect, mode=train, model=yolov5n.yaml, data=wisard_dataset_rgb.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train103, exist_ok=False, pretrained=yolov5nu.pt, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

train: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/rgb/labels.cache... 426 images, 12 backgrounds, 0 corrupt: 100%|██████████| 426/426 [00:00<?, ?it/s]
val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/labels.cache... 141 images, 4 backgrounds, 0 corrupt: 100%|██████████| 141/141 [00:00<?, ?it/s]


optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 512 train, 512 val
Using 0 dataloader workers
Logging results to runs/detect/train103
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        141        357    0.00221     0.0448     0.0013   0.000325

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/2      3.77G      2.338      1.475      1.235         60        512: 100%|██████████| 27/27 [00:28<00:00,  1.05s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:15<00:00,  3.04s/it]


                   all        141        357      0.048      0.213     0.0485     0.0154

2 epochs completed in 0.028 hours.
Optimizer stripped from runs/detect/train117/weights/last.pt, 5.2MB
Optimizer stripped from runs/detect/train117/weights/best.pt, 5.2MB

Validating runs/detect/train117/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 MPS (Apple M4 Pro)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:06<00:00,  1.32s/it]


                   all        141        357     0.0479      0.213     0.0484     0.0154
Speed: 0.8ms preprocess, 6.2ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/train117
💡 Learn more at https://docs.ultralytics.com/modes/train
Saved runs/detect/tune23/tune_scatter_plots.png
Saved runs/detect/tune23/tune_fitness.png

Tuner: 15/15 iterations complete ✅ (1936.06s)
Tuner: Results saved to runs/detect/tune23
Tuner: Best fitness=0.07113 observed at iteration 13
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.28846, 'metrics/recall(B)': 0.16807, 'metrics/mAP50(B)': 0.19777, 'metrics/mAP50-95(B)': 0.05706, 'val/box_loss': 3.50608, 'val/cls_loss': 3.69979, 'val/dfl_loss': 1.83761, 'fitness': 0.07113}
Tuner: Best fitness model is runs/detect/train115
Tuner: Best fitness hyperparameters are printed below.

Printing 'runs/detect/tune23/best_hyperparameters.yaml'

lr0: 0.00977
momentum: 0.85



### Manual Random Search

In [26]:
import random

search_space = {
    "lr0": (1e-5,1e-4,1e-3,1e-2,1e-1),
    "batch": [4,6,8,10,12,14,16,18,20]
}

#Save the hyperparameter values used in the tuning
lr0_dict = {}
batch_dict = {}

for i in range(15):
  lr0 = search_space["lr0"][random.randint(0,4)]
  batch = search_space["batch"][random.randint(0,8)]

  lr0_dict[i] = lr0
  batch_dict[i] = batch

  modelv5_tuning.train(
    data='wisard_dataset_rgb.yaml',
    epochs=5,
    imgsz=640,
    project='runs/v5_RGB',
    batch=batch,
    lr0=lr0,
    optimizer = 'SGD',
    device='mps',
    workers=4
  )


New https://pypi.org/project/ultralytics/8.3.126 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 MPS (Apple M4 Pro)
engine/trainer: task=detect, mode=train, model=yolov5nu.pt, data=wisard_dataset_rgb.yaml, epochs=5, time=None, patience=100, batch=14, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=4, project=runs/v5_RGB, name=train43, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/rgb/labels... 1128 images, 115 backgrounds, 0 corrupt: 100%|██████████| 1128/1128 [00:00<00:00, 5637.98it/s]

train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00001.jpg: 1 duplicate labels removed
train: New cache created: /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/rgb/labels.cache



val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/labels... 375 images, 59 backgrounds, 0 corrupt: 100%|██████████| 375/375 [00:00<00:00, 5453.66it/s]

val: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00604.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00605.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00606.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00607.jpg: 1 duplicate labels removed
val: New cache created: /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/labels.cache
Plotting labels to 

...


val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/labels.cache... 375 images, 59 backgrounds, 0 corrupt: 100%|██████████| 375/375 [00:00<?, ?it/s]

val: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00604.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00605.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00606.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/images/210924_FHL_Enterprise_VIS_0401.mp4_00607.jpg: 1 duplicate labels removed
Plotting labels to runs/v5_RGB/train432222/labels.jpg... 


optimizer: SGD(lr=1e-05, momentum=0.937) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.000515625), 75 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/v5_RGB/train432222
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      15.8G      2.405      1.504      1.192         50        640:  50%|█████     | 94/188 [00:29<00:29,  3.14it/s]


KeyboardInterrupt: 

In [18]:
import pandas as pd

result_paths = ['train43', 'train432', 'train4322', 'train43222']

summary_rows = []

for i, path in enumerate(result_paths):
    results_path = f'runs/v5_RGB/{path}/results.csv'
    df = pd.read_csv(results_path)
    
    # Get the 5th epoch (index 4)
    if len(df) >= 5:
        recall = df.loc[4, 'metrics/recall(B)']
    else:
        recall = None  # fallback if file is too short

    # Build summary row
    summary_rows.append({
        'run_name': path,
        'lr0': lr0_dict[i],
        'batch': batch_dict[i],
        'recall': recall
    })

print(summary_rows)
# Create summary DataFrame
results_df = pd.DataFrame(summary_rows)

# Sort by recall
results_df_sorted = results_df.sort_values(by='recall', ascending=False)

NameError: name 'lr0_dict' is not defined

In [48]:
print(summary_rows)

[{'run_name': 'train43', 'lr0': 0.1, 'batch': 14, 'recall': np.float64(0.14223)}, {'run_name': 'train432', 'lr0': 1e-05, 'batch': 6, 'recall': np.float64(0.19435)}, {'run_name': 'train4322', 'lr0': 1e-05, 'batch': 6, 'recall': np.float64(0.19281)}, {'run_name': 'train43222', 'lr0': 1e-05, 'batch': 6, 'recall': np.float64(0.197)}]


In [14]:
import os
from pathlib import Path

# Root folder of your dataset
root = Path('/Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered')

# Subfolders to check
splits = ['train', 'val', 'test']
modes = ['thermal', 'rgb']  # Optional: remove 'rgb' if you're only using thermal
image_exts = ('.jpg', '.jpeg', '.png')

# Count images
for mode in modes:
    print(f"\n {mode.upper()} images:")
    for split in splits:
        image_dir = root / split / mode / 'images'
        if not image_dir.exists():
            print(f"  ❌ {split} folder not found: {image_dir}")
            continue
        count = len([f for f in image_dir.iterdir() if f.suffix.lower() in image_exts])
        print(f"  ✅ {split}: {count} images")



 THERMAL images:
  ✅ train: 1128 images
  ✅ val: 375 images
  ✅ test: 374 images

 RGB images:
  ✅ train: 1128 images
  ✅ val: 375 images
  ✅ test: 375 images


## Training

In [11]:
modelv5.train(
    data='wisard_dataset_thermal.yaml',
    epochs=30,
    lr0=0.0001,
    batch=8,
    imgsz=640,
    device="mps",
    project="runs/v5_thermal"
)

New https://pypi.org/project/ultralytics/8.3.126 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 MPS (Apple M4 Pro)
engine/trainer: task=detect, mode=train, model=yolov5nu.pt, data=wisard_dataset_thermal.yaml, epochs=30, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=runs/v5_thermal, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/labels.cache... 1128 images, 100 backgrounds, 0 corrupt: 100%|██████████| 1128/1128 [00:00<?, ?it/s]

train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00001.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00002.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00003.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00004.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/thermal/images/DJI_0402.mp4_00005.jpg: 3 duplicate labels removed



val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/thermal/labels.cache... 375 images, 35 backgrounds, 0 corrupt: 100%|██████████| 375/375 [00:00<?, ?it/s]

Plotting labels to runs/v5_thermal/train2/labels.jpg... 


...

      30/30      12.4G      1.038     0.5036     0.8767         26        640: 100%|██████████| 141/141 [00:35<00:00,  3.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:07<00:00,  3.34it/s]


                   all        375       1023      0.691      0.518      0.553      0.254

30 epochs completed in 0.683 hours.
Optimizer stripped from runs/v5_thermal/train2/weights/last.pt, 5.3MB
Optimizer stripped from runs/v5_thermal/train2/weights/best.pt, 5.3MB

Validating runs/v5_thermal/train2/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 MPS (Apple M4 Pro)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:20<00:00,  1.15it/s]


                   all        375       1023      0.764      0.521      0.578      0.275
Speed: 1.3ms preprocess, 13.1ms inference, 0.0ms loss, 5.3ms postprocess per image
Results saved to runs/v5_thermal/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x391dc92e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

Validate the best model

In [16]:
# Evaluate on test set defined in the YAML file
metrics = modelv5.val(data="wisard_dataset_thermal.yaml", split="test")

Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 MPS (Apple M4 Pro)


val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/thermal/labels... 374 images, 105 backgrounds, 0 corrupt: 100%|██████████| 374/374 [00:00<00:00, 5529.11it/s]

val: New cache created: /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/thermal/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:14<00:00,  3.29it/s]


                   all        374        763      0.737      0.523      0.546      0.262
Speed: 2.2ms preprocess, 21.8ms inference, 0.0ms loss, 5.9ms postprocess per image
Results saved to runs/v5_thermal/train25


Use the model for inference

In [17]:
modelv5.predict(
    source="datasets/WiSARD_dataset/registered/test/thermal/images",
    save=True,
    conf=0.25
)


image 1/374 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/thermal/images/210417_MtErie_Enterprise_IR_0004_00000212.jpeg: 512x640 3 persons, 3960.0ms
image 2/374 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/thermal/images/210417_MtErie_Enterprise_IR_0004_00000213.jpeg: 512x640 2 persons, 11.2ms
image 3/374 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/thermal/images/210417_MtErie_Enterprise_IR_0004_00000214.jpeg: 512x640 2 persons, 7.1ms
image 4/374 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/thermal/images/210417_MtErie_Enterprise_IR_0004_00000215.jpeg: 512x640 3 persons, 8.6ms
image 5/374 /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/test/thermal/images/210417_MtErie_Enterpris

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person'}
 obb: None
 orig_img: array([[[ 90,  90,  90],
         [ 87,  87,  87],
         [ 78,  78,  78],
         ...,
         [128, 128, 128],
         [123, 123, 123],
         [126, 126, 126]],
 
        [[ 89,  89,  89],
         [ 86,  86,  86],
         [ 81,  81,  81],
         ...,
         [123, 123, 123],
         [123, 123, 123],
         [123, 123, 123]],
 
        [[ 82,  82,  82],
         [ 79,  79,  79],
         [ 79,  79,  79],
         ...,
         [119, 119, 119],
         [123, 123, 123],
         [122, 122, 122]],
 
        ...,
 
        [[122, 122, 122],
         [125, 125, 125],
         [125, 125, 125],
         ...,
         [170, 170, 170],
         [168, 168, 168],
         [181, 181, 181]],
 
        [[121, 121, 121],
         [123, 123, 123],
         [123, 123, 123],
         ...,
         [172, 172

# VOLOv8

## Hyperparameter Tuning

In [38]:
# Define search space
search_space = {
    "lr0": (1e-5, 1e-1),
    "batch": [5, 10, 20]
}

# Tune hyperparameters on WISARD DATASET
modelv8.tune(
    data="wisard_dataset_rgb.yaml",
    epochs=5,
    iterations=3,
    optimizer="Adam",
    space=search_space,
    device="cpu"
)

Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune12'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/3 with hyperparameters: {'lr0': 0.01, 'batch': 10}


/Users/ikmalbasirun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Transferred 355/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.3.121 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.91 🚀 Python-3.9.6 torch-2.6.0 CPU (Apple M4 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=wisard_dataset_rgb.yaml, epochs=5, time=None, patience=100, batch=10, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train67, exist_ok=False, pretrained=yolov8n.pt, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fals

train: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/train/rgb/labels.cache... 453 images, 11 backgrounds, 0 corrupt: 100%|██████████| 453/453 [00:00<?, ?it/s]
val: Scanning /Users/ikmalbasirun/Documents/GitHub/G5_image_classification_project/datasets/WiSARD_dataset/registered/val/rgb/labels.cache... 113 images, 1 backgrounds, 0 corrupt: 100%|██████████| 113/113 [00:00<?, ?it/s]


optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train67
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        1/5         0G      2.402      2.175      1.365         10        640: 100%|██████████| 46/46 [01:18<00:00,  1.70s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:11<00:00,  1.91s/it]
  0%|          | 0/46 [00:00<?, ?it/s]

                   all        113        200      0.764      0.065     0.0889     0.0226

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/5         0G       2.56      1.716      1.407         53        640:  72%|███████▏  | 33/46 [00:57<00:22,  1.77s/it]

KeyboardInterrupt: 

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
